# Imports

In [44]:
from transformers import GPT2TokenizerFast, GPT2DoubleHeadsModel
import torch

# Setting Up Model

In [64]:
tokenizer = GPT2TokenizerFast.from_pretrained('gpt2-large')
model = GPT2DoubleHeadsModel.from_pretrained('gpt2-large', pad_token_id=tokenizer.eos_token_id)

KeyboardInterrupt: 

In [60]:
inputs = tokenizer.encode("One does not simply", return_tensors='pt')

In [61]:
output = model.generate(inputs, max_length=50, do_sample=True, top_p=0.95, top_k=50)

In [62]:
print(tokenizer.decode(output[0], skip_special_tokens=True))

One does not simply mean "don't be afraid". Rather, it also means "don't be afraid of being someone else, but who is not."

As we learn more and more about our lives through yoga, we begin to understand how
